In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [5]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4') 
#unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\harit\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
data = pd.read_csv('D:/Projects/Toxic-Comment-Classification/archive/train.csv')
#data.sample(5)
test_data = pd.read_csv('D:/Projects/Toxic-Comment-Classification/archive/test.csv')
test_labels = pd.read_csv('D:/Projects/Toxic-Comment-Classification/archive/test_labels.csv')
merged_data = pd.merge(test_data, test_labels, on='id')

# Filter rows where toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate' are not -1
filtered_test_data = merged_data[
    (merged_data['toxic'] != -1) &
    (merged_data['severe_toxic'] != -1) &
    (merged_data['obscene'] != -1) &
    (merged_data['threat'] != -1) &
    (merged_data['insult'] != -1) &
    (merged_data['identity_hate'] != -1)
]
filtered_test_data.to_csv('D:/Projects/Toxic-Comment-Classification/archive/cleaned_test_data.csv', index=False)
filtered_test_data.sample(200)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
81071,87506a7df3cc51b0,""" \n\n == Unjustly affected by autoblock == \n...",0,0,0,0,0,0
65403,6cd315df039e7c35,"Reveille is one of my fav bands, the other ban...",0,0,0,0,0,0
40995,4413d8ab5d8542a3,Mgfrank \n\n \n A tag has been placed on Mgf...,0,0,0,0,0,0
4620,07db09d915da9dfb,== KochFacts.com == \n\n :Undid revision 63429...,0,0,0,0,0,0
83206,8abf37ff0380fed7,b HELPPPP MEEEE EMEE,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
3584,062e6a5793db2720,== Pictures of penis sizes == \n\n Shouldn't w...,1,0,0,0,0,0
62542,681cade4f72def7f,""" \n\n == Citations? == \n\n I notice that thi...",0,0,0,0,0,0
78888,83aa72a87f749fdc,LENNIE IS INJURED FOR THE REMAINDER OF THE SEA...,1,1,1,0,1,0
8725,0e9f354ae15a851c,== Thanks == \n\n Hi! Thanks once again for no...,0,0,0,0,0,0


In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [8]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation and numbers
    text = text.strip()  # Remove leading and trailing whitespaces
    tokens = text.split()  # Tokenize the text by splitting on whitespace
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize the tokens
    cleaned_text = ' '.join(tokens)  # Join tokens back into a single string
    return cleaned_text

In [9]:
data['cleaned_comment'] = data['comment_text'].apply(clean_text)
data['is_toxic'] = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
filtered_test_data['cleaned_comment'] = filtered_test_data['comment_text'].apply(clean_text)
filtered_test_data['is_toxic'] = filtered_test_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)


C:\Users\harit\AppData\Local\Temp\ipykernel_25928\3316149774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_data['cleaned_comment'] = filtered_test_data['comment_text'].apply(clean_text)
C:\Users\harit\AppData\Local\Temp\ipykernel_25928\3316149774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_data['is_toxic'] = filtered_test_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)


In [10]:
data['is_toxic'].unique()


array([0, 1], dtype=int64)

In [11]:
filtered_test_data['is_toxic'].unique()

array([0, 1], dtype=int64)

In [12]:
data.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_comment,is_toxic
26651,469c268d8e50ad8d,"""\n\n punt again \n\nSee this diff by Taharqa:...",0,0,0,0,0,0,punt see diff taharqa httpenwikipediaorgwindex...,0
46637,7ca5001fa132fe26,Thanks and a Suggestion \n\nThank you for your...,0,0,0,0,0,0,thanks suggestion thank kind word support thin...,0
77223,ced7843a6992d2c5,...but if you haven't had enough of the ongoin...,0,0,0,0,0,0,havent enough ongoing debate try wwwsmhcomauht...,0
16953,2cba39774aae8334,Lyrics \n\nThe whole lyrics section is pointle...,0,0,0,0,0,0,lyric whole lyric section pointless removed,0
23727,3eafef6395b295a9,Thanks for letting me know about signing. I'll...,0,0,0,0,0,0,thanks letting know signing ill try harder ive...,0


In [13]:

filtered_test_data.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_comment,is_toxic
147012,f5c13312d5ff29be,Hi cadondondo you are an asome riter you sare ...,0,0,0,0,0,0,hi cadondondo asome riter sare rare desise lik...,0
85044,8dd3e985aee86252,(we kick them) - like atheist talk about relig...,0,0,0,0,0,0,kick like atheist talk religion without religi...,0
20162,21ca82d43fca45a3,== Lenght == \n\n In the article it states tha...,0,0,0,0,0,0,lenght article state plane long spec correct s...,0
34515,39631827dda9d1bb,"*Now, now! Has working on Wikipedia taught yo...",0,0,0,0,0,0,working wikipedia taught nothing sourcing repo...,0
121245,ca8311c7fdccdd47,""" \n\n ==Pre-Trial== \n\n I would think that t...",0,0,0,0,0,0,pretrial would think information pretrial case...,0


In [14]:
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(data['cleaned_comment'])
y = data['is_toxic']
X_test = vectorizer.transform(filtered_test_data['cleaned_comment'])
y_test= filtered_test_data['is_toxic']

In [15]:
# Count occurrences of each distinct value in y_test
label_counts = y_test.value_counts()

# Print the counts
print("Label Distribution in Test Data:")
print(label_counts)

Label Distribution in Test Data:
is_toxic
0    57735
1     6243
Name: count, dtype: int64


In [16]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X,y)


In [17]:
nb = MultinomialNB()
nb.fit(X_train_resampled,y_train_resampled)

MultinomialNB()

In [18]:
y_pred = nb.predict(X_test)

In [19]:
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
y_prob = nb.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
ks_statistic, _ = ks_2samp(y_prob[y_test == 1], y_prob[y_test == 0])

In [20]:
print("auc",auc)
print(f"KS Statistic: {ks_statistic:.2f}")

auc 0.944974412564901
KS Statistic: 0.75


In [21]:
print(classification_report(filtered_test_data['is_toxic'], y_pred))

              precision    recall  f1-score   support

           0       0.99      0.84      0.91     57735
           1       0.38      0.91      0.54      6243

    accuracy                           0.85     63978
   macro avg       0.68      0.87      0.72     63978
weighted avg       0.93      0.85      0.87     63978



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from scipy.stats import ks_2samp

# Define the parameter grid with different alpha values
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Numeric values for alpha
}

# Initialize MultinomialNB classifier
nb = MultinomialNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, cv=5, scoring='f1')

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Print accuracy for all alpha values from cv_results_
print("Accuracy for all alpha values:")
for i in range(len(grid_search.cv_results_['params'])):
    alpha = grid_search.cv_results_['params'][i]['alpha']
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    print(f"Alpha: {alpha}, F1-score: {mean_score:.4f}")

# Print the best parameters and the best score
print("\nBest parameters found: ", grid_search.best_params_)
print("Best F1 score found: ", grid_search.best_score_)

# Predict with the best model
best_nb = grid_search.best_estimator_
y_pred_best = best_nb.predict(X_test)

# Evaluate the best model
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

# Calculate and print accuracy for the best model
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"\nAccuracy of the best model: {accuracy_best:.4f}")

# Print ROC AUC and KS Statistic for the best model
y_prob_best = best_nb.predict_proba(X_test)[:, 1]
auc_best = roc_auc_score(y_test, y_prob_best)  # Corrected here
ks_statistic_best, _ = ks_2samp(y_prob_best[y_test == 1], y_prob_best[y_test == 0])
print("Best Model ROC AUC:", auc_best)
print(f"Best Model KS Statistic: {ks_statistic_best:.2f}")


Accuracy for all alpha values:
Alpha: 0.1, F1-score: 0.8964
Alpha: 0.5, F1-score: 0.8968
Alpha: 1.0, F1-score: 0.8967
Alpha: 1.5, F1-score: 0.8967
Alpha: 2.0, F1-score: 0.8966

Best parameters found:  {'alpha': 0.5}
Best F1 score found:  0.8967538267563129

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.84      0.91     57735
           1       0.38      0.90      0.54      6243

    accuracy                           0.85     63978
   macro avg       0.69      0.87      0.72     63978
weighted avg       0.93      0.85      0.87     63978


Accuracy of the best model: 0.8489
Best Model ROC AUC: 0.9437141515011925
Best Model KS Statistic: 0.75


: 